In [2]:
!pip install -qr requirements.txt 

In [1]:
from ouster import client
from ouster import pcap
from contextlib import closing
from os import listdir
from os.path import join, isdir, abspath
from pathlib import Path
import subprocess

from tqdm import tqdm
import cv2
import numpy as np
from joblib import Parallel, delayed

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
def transform_pcap(meta_path,file_path,save_path,clip,channel):
    
  print(f"Started {file_path}")

  with open(meta_path, 'r') as f:
    metadata = client.SensorInfo(f.read())

  pcap_file = pcap.Pcap(file_path, metadata)

  fps = int(str(metadata.mode)[-2:])
  width = int(str(metadata.mode)[:4])
  height = int(str(metadata.prod_line).split('-')[2])

  if clip:
      left_clip = 190#0
      right_clip = 275#-1024
  else:
      left_clip = 0
      right_clip = -1024

  width -= left_clip + right_clip
  
  base = join(save_path, channel)
  Path(base).mkdir(parents=True,exist_ok=True)
  # print(meta_path,file_path,save_path)

  with closing(client.Scans(pcap_file)) as scans:

    vid_writer_sig = cv2.VideoWriter(join(save_path,"signal.avi"), cv2.VideoWriter_fourcc(*"RGBA"), fps, (width, height))
    # vid_writer_range = cv2.VideoWriter(save_path+"_range.avi", cv2.VideoWriter_fourcc(*"RGBA"), fps, (width, height))
    # vid_writer_ref = cv2.VideoWriter(save_path+"_reflectivity.avi", cv2.VideoWriter_fourcc(*"RGBA"), fps, (width, height))
    # vid_writer_nir = cv2.VideoWriter(save_path+"_near_ir.avi", cv2.VideoWriter_fourcc(*"RGBA"), fps, (width, height))


    for i,scan in enumerate(tqdm(scans, position=0, leave=True)):

      ref_field = scan.field(client.ChanField.REFLECTIVITY)
      ref_val = client.destagger(pcap_file.metadata, ref_field)
      ref_img = ref_val.astype(np.uint8)[:,left_clip:-right_clip]

      sig_field = scan.field(client.ChanField.SIGNAL)
      sig_val = client.destagger(pcap_file.metadata, sig_field)
      sig_img = sig_val.astype(np.uint8)[:,left_clip:-right_clip]

      ir_field = scan.field(client.ChanField.NEAR_IR)
      ir_val = client.destagger(pcap_file.metadata, ir_field)
      ir_img = ir_val.astype(np.uint8)[:,left_clip:-right_clip]

      range_field = scan.field(client.ChanField.RANGE)
      range_val = client.destagger(pcap_file.metadata, range_field)
      range_img = range_val.astype(np.uint8)[:,left_clip:-right_clip]

      custom_img = np.dstack((sig_img,range_img,ir_img))

      sig_img = np.dstack((sig_img, sig_img, sig_img))
      range_img = np.dstack((range_img, range_img, range_img))
      ref_img = np.dstack((ref_img, ref_img, ref_img))
      ir_img = np.dstack((ir_img, ir_img, ir_img))


      # sig_img = cv2.cvtColor(sig_img, cv2.COLOR_BGR2HSV)
      # sig_img = cv2.applyColorMap(sig_img, cv2.COLORMAP_TURBO)
        
      
      vid_writer_sig.write(sig_img)
      # plt.imshow(sig_img)
      # plt.show()
      # if channel == "ir":
      #   cv2.imwrite(join(base,f"{i+1}.png"),ir_img)
      # elif channel == "signal":
      #   cv2.imwrite(join(base,f"{i+1}.png"),sig_img)
      # elif channel == "reflection":
      #     cv2.imwrite(join(base,f"{i+1}.png"),ref_img)
      # elif channel == "range":
      #     cv2.imwrite(join(base,f"{i+1}.png"),range_img)
      # else:
      #   cv2.imwrite(join(base,f"{i+1}.png"),custom_img)

      # vid_writer_range.write(range_img)
      # vid_writer_ref.write(ref_img)    
      # vid_writer_nir.write(ir_img)

    vid_writer_sig.release()
    # vid_writer_range.release()
    # vid_writer_ref.release()
    # vid_writer_nir.release()
    cv2.destroyAllWindows()
    
  print(f"Finished {file_path}")

In [4]:
def recurse_transform(base,transform_fun,clip,channel):

    try:
        files = listdir(base)
    except FileNotFoundError:
        print(f"{base} does not exist")
        return

    directories = list(filter(lambda x: isdir(join(base,x)),files))
    
    pcap_files = list(filter(lambda x: x.endswith(".pcap"),files))
    
    # meta_files = map(lambda x: str(Path(x).with_suffix(".json")),pcap_files)
    # meta_files = map(lambda x: join(base,"meta.json"),pcap_files)

    base_parent = Path(base).parent.absolute()
    p_files = list(filter(lambda x: x.endswith(".pcap"),listdir(str(base_parent))))
    meta_files = [join(base_parent,str(Path(p_files[0]).with_suffix(".json")))] * len(p_files)


    if pcap_files:
      print(f"Started {base} :: {pcap_files}")
        
      Parallel(n_jobs=4)(delayed(transform_fun)(meta,join(base,pcap),join(base,Path(pcap).stem),clip,channel)for meta,pcap in zip(meta_files,pcap_files))

      print(f"Finished {pcap_files}")
    
    for directory in directories:
        recurse_transform(join(base,directory),transform_fun,clip,channel)

In [ ]:
x = [
 'SBIR_2022-10-06',
 'SBIR_2022-10-07',
 'SBIR_2022-10-08',
 'SBIR_2022-10-20',
 'SBIR_2022-10-21',
 'SBIR_2022-10-22']

for path in x:
  folders = listdir(join(".","data","SBIR",path))
  for folder in folders:
    if folder != "extracted":
      recurse_transform(join(".","data","SBIR",path,folder,"small_files"),transform_pcap,clip=True,channel="signal")

Started .\data\SBIR\SBIR_2022-10-06\14\small_files :: ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Started .\data\SBIR\SBIR_2022-10-06\1\small_files :: ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Started .\data\SBIR\SBIR_2022-10-06\2\small_files :: ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap']
Started .\data\SBIR\SBIR_2022-10-06\3\small_files :: ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap', '7.pcap', '8.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap', '7.pcap', '8.pcap']
Started .\data\SBIR\SBIR_2022-10-06\4\small_files :: ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap', '7.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap', '7.pcap']
Started .\data\SBIR\SBIR_2022-10-06\5\small_files :: ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pc

In [5]:
recurse_transform(join(".","data","SBIR","SBIR_2022-10-06", "2","small_files"),transform_pcap,clip=True,channel="ir")

Started ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap']


In [8]:
recurse_transform(join(".","data","from_car","OS1-64_2022-11-16","processed"),transform_pcap,clip=True,channel="ir")

Started ['1.pcap', '2.pcap', '3.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap']
Started ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Started ['1.pcap', '2.pcap', '3.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap']


In [9]:
recurse_transform(join(".","data","from_car","OS1-64_2022-11-21","processed"),transform_pcap,clip=True,channel="ir")

Started ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap']
Started ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap']
Started ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap', '7.pcap']
Finished ['1.pcap', '2.pcap', '3.pcap', '4.pcap', '5.pcap', '6.pcap', '7.pcap']


<h1>Old Code</h1>

In [ ]:
prefix = ".\\SBIR\\"
dates = {"SBIR_2022-09-24\\":(1,18),"SBIR_2022-10-08\\":(21,23),"SBIR_2022-10-07\\":(15,20),"SBIR_2022-10-06\\":(1,14),"SBIR_2022-10-21\\":(37,40),"SBIR_2022-10-22\\":(41,53),"SBIR_2022-10-20\\":(33,36)}

In [11]:
def process_given_date(date,val):
  prefix = "D:\\SBIR\\" + date + f"{val}\\"

  save_prefix = "D:\\SBIR\\" + date  + f"extracted\\{val}_test\\" 
    
  Path(save_prefix).mkdir(parents=True,exist_ok=True)

  files = [f for f in listdir(prefix)]

  if not files:
    return

  meta_path = prefix + files[0][:-5] + ".json"

  print(f"Started {prefix}...")

  for i in filter(lambda e: e.endswith(".pcap"), listdir(prefix + "small_files\\")):

    i = i[:-5]

    pcap_path = prefix + "small_files\\" + str(i) + ".pcap"

    save_path = save_prefix + str(i)

    print(f"Processing {pcap_path}")

    transform_pcap(meta_path,pcap_path,save_path)
    
  print(f"Finished {prefix}...")


In [12]:
process_given_date("SBIR_2022-09-24\\",9)

Started D:\SBIR\SBIR_2022-09-24\9\...
Processing D:\SBIR\SBIR_2022-09-24\9\small_files\1.pcap


783it [00:19, 40.45it/s]


Processing D:\SBIR\SBIR_2022-09-24\9\small_files\2.pcap


3601it [01:31, 39.52it/s]


Processing D:\SBIR\SBIR_2022-09-24\9\small_files\3.pcap


...

Processing D:\SBIR\SBIR_2022-09-24\9\small_files\6.pcap


3602it [01:29, 40.03it/s]


Processing D:\SBIR\SBIR_2022-09-24\9\small_files\7.pcap


489it [00:12, 40.73it/s]

Finished D:\SBIR\SBIR_2022-09-24\9\...


In [13]:
date = "SBIR_2022-10-08\\"

In [14]:
for d,v in [(date,dates[date])]:
    Parallel(n_jobs=4)(delayed(process_given_date)(d,val) for val in range(v[0],v[1]+1))

In [ ]:
process_given_date("SBIR_2022-10-06\\",2)

Started D:\SBIR\SBIR_2022-10-06\2\...
Processing D:\SBIR\SBIR_2022-10-06\2\small_files\1.pcap


3556it [00:55, 64.55it/s]


Processing D:\SBIR\SBIR_2022-10-06\2\small_files\2.pcap


3601it [00:55, 64.94it/s]


Processing D:\SBIR\SBIR_2022-10-06\2\small_files\3.pcap


3601it [00:54, 65.97it/s]


Processing D:\SBIR\SBIR_2022-10-06\2\small_files\4.pcap


3601it [00:54, 66.61it/s]


Processing D:\SBIR\SBIR_2022-10-06\2\small_files\5.pcap


2937it [00:44, 70.97it/s]